TODO: Introduction

In [8]:
from bleu import compute_bleu
from data_loader import tokenize_de, tokenize_en
from device import select_device, with_cpu, with_gpu
from main import evaluate_batch
import json
from parse import get_config
import torch
import torchtext
from utils import list2words, torch2words

# Model

Load weights of model and parses model configuration.

In [2]:
config_name = 'final'
config_path = f'configs/{config_name}.json'

In [10]:
use_gpu, device, device_idx = select_device()
config = get_config(use_gpu, device, device_idx, load_weights=True, config_path=config_path, parse_args=False)
source_language = config.get('src_language')
target_language = config.get('trg_language')

# Load Test Data

In [13]:
csv_dir_path = '.data/multi30k'
EOS_token = config.get('EOS_token')
PAD_token = config.get('PAd_token')
SOS_token = config.get('SOS_token')
source_field = torchtext.data.Field(
    tokenize=tokenize_de,
    init_token=SOS_token,
    eos_token=EOS_token,
    pad_token=PAD_token,
    include_lengths=True
)
target_field = torchtext.data.Field(
    tokenize=tokenize_en,
    init_token=SOS_token,
    eos_token=EOS_token,
    pad_token=PAD_token,
    include_lengths=True
)
data_fields = [('src', source_field), ('trg', target_field)]
train, val, test = torchtext.data.TabularDataset.splits(
    path=csv_dir_path,
    test='test.csv',
    train='train.csv',
    validation='val.csv',
    format='csv',
    fields=data_fields,
    skip_header=True
)
source_vocabulary_size = config.get('source_vocabulary_size')
target_vocabulary_size = config.get('target_vocabulary_size')
source_field.build_vocab(train, val, max_size=source_vocabulary_size)
target_field.build_vocab(train, val, max_size=target_vocabulary_size)
_, _, test_iter = torchtext.data.BucketIterator.splits(
    (train, val, test),
    batch_size=config.get('batch_size'),
    device=device,
    shuffle=True,
    sort_key=lambda x: len(x.src)
)

# BLEU

In [14]:
reference_corpus = []
translation_corpus = []
for test_batch in test_iter:
    test_loss, translations = evaluate_batch(config, test_batch)
    test_batch_trg, _ = test_batch.trg
    _, batch_size = test_batch_trg.shape
    references = map(lambda i: torch2words(target_language, test_batch_trg[:, i]), range(batch_size))
    references = map(lambda words: [list(filter_words(words, SOS_token, EOS_token, PAD_token))], references)
    reference_corpus.extend(references)
    translations = map(lambda translation: list2words(target_language, translation), translations)
    translations = map(lambda words: list(filter_words(words, SOS_token, EOS_token, PAD_token)), translations)
    translation_corpus.extend(translations)
bleu = compute_bleu(reference_corpus, translation_corpus)
print(f'BLEU on test set: {bleu}')

RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.

# Attention

# Learning Curves